# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Instalar selenium:

!pip install selenium

In [ ]:
#chromedriver --> Chrome
# geckodriver --> Firefox

from selenium import webdriver                  # Criar o navegador
from selenium.webdriver.common.by import By     # Localizar elementos (os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no teclado

navegador = webdriver.Chrome()

# Passo 1: Entrar no Google
navegador.get('https://www.google.com/')

# Passo 2: Pesquisar a cotação do dólar
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Passo 3: Pegar a cotação do dólar
cotacao_dolar = navegador.find_element(By.XPATH, 
                                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do dólar: {cotacao_dolar}')

# Passo 4: Pegar a cotação do euro
navegador.get('https://www.google.com/')

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, 
                                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do euro: {cotacao_euro}')

# Passo 5: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(f'Cotação do ouro: {cotacao_ouro}')

# Fechar automaticamente o navegador quando terminar tudo
navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# Passo 6: Atualizar a minha base de dados com as novas cotações
# Atualizar a cotação de acordo com a moeda correspondente

# dolar
# as linhas onde a coluna 'Moeda' == 'Dólar'
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)

# euro
# as linhas onde a coluna 'Moeda' == 'Euro'
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)

# ouro
# as linhas onde a coluna 'Moeda' == 'Ouro'
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Atualizar preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Atualizar preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
tabela.to_excel('Produtos_Novos.xlsx', index=False)
# index=False --> Salva sem a coluna de índices